In [16]:
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [3]:
from google.colab import files
uploaded = files.upload()

Saving main_dataset.xlsx to main_dataset.xlsx


In [30]:
# Step 1: Data Collection – Load dataset from Excel
file_path = "main_dataset.xlsx"
user_data = pd.read_excel(file_path, sheet_name="user_data")
job_data = pd.read_excel(file_path, sheet_name="job_data")
interaction_data = pd.read_excel(file_path, sheet_name="interaction_data")

In [31]:
# Step 2: Data Preprocessing – Handle missing values, combine attributes
user_data.fillna('', inplace=True)
job_data.fillna('', inplace=True)
interaction_data.fillna(0, inplace=True)

user_data['Profile'] = (
    user_data['Skills'] + ' ' +
    user_data['Interests'] + ' ' +
    user_data['Previous Jobs'] + ' ' +
    user_data['Looking Jobs'] + ' ' +
    user_data['Description']
)

job_data['Details'] = (
    job_data['Job Title'] + ' ' +
    job_data['Skills Required'] + ' ' +
    job_data['Experience Required'] + ' ' +
    job_data['Job Description']
)

In [32]:
# Step 3: Feature Engineering – TF-IDF vectorization for content-based filtering
combined_text = pd.concat([user_data['Profile'], job_data['Details']], axis=0)
tfidf = TfidfVectorizer(stop_words="english", max_features=5000)  # Limit features to optimize performance
tfidf_matrix = tfidf.fit_transform(combined_text)

# Reduce dimensionality of TF-IDF matrix
svd_tfidf = TruncatedSVD(n_components=200, random_state=42)
reduced_tfidf_matrix = svd_tfidf.fit_transform(tfidf_matrix)

# Split back into user and job matrices
user_tfidf = reduced_tfidf_matrix[:len(user_data)]
job_tfidf = reduced_tfidf_matrix[len(user_data):]

In [33]:
# Step 4: Model Training (Content-Based Filtering) – Compute cosine similarity
similarity_matrix = cosine_similarity(user_tfidf, job_tfidf)

In [34]:
# Step 5: Model Training (Collaborative Filtering) – SVD on interaction matrix
interaction_matrix = interaction_data.pivot_table(
    index='User ID', columns='Job ID', values='Interaction Value', fill_value=0
)

In [35]:
# Step 6: Train-Test Split
train_data, test_data = train_test_split(interaction_data, test_size=0.2, random_state=42)
train_interaction_matrix = train_data.pivot_table(
    index='User ID', columns='Job ID', values='Interaction Value', fill_value=0
)
test_interaction_matrix = test_data.pivot_table(
    index='User ID', columns='Job ID', values='Interaction Value', fill_value=0
)

train_user_ids = train_interaction_matrix.index.tolist()
train_job_ids = train_interaction_matrix.columns.tolist()

In [36]:
# Step 7: Collaborative Filtering using SVD
# Hyperparameter Tuning – Optimize n_components
n_components = min(100, train_interaction_matrix.shape[1])  # Dynamically adjust components
svd = TruncatedSVD(n_components=n_components, random_state=42)
latent_matrix = svd.fit_transform(train_interaction_matrix)
predicted_train_matrix = np.dot(latent_matrix, svd.components_)

In [37]:
# Step 8: Hybrid Model
# Normalize Scores
scaler = MinMaxScaler()
content_scores = scaler.fit_transform(similarity_matrix)
collab_scores = scaler.fit_transform(predicted_train_matrix)

# Find common user IDs in both matrices
common_user_ids = list(set(user_data['User ID']).intersection(set(train_user_ids)))
user_indices_content = [list(user_data['User ID']).index(user_id) for user_id in common_user_ids]
user_indices_collab = [train_user_ids.index(user_id) for user_id in common_user_ids]

# Find common job IDs in both matrices
common_job_ids = list(set(job_data['Job ID']).intersection(set(train_job_ids)))
job_indices_content = [list(job_data['Job ID']).index(job_id) for job_id in common_job_ids]
job_indices_collab = [train_job_ids.index(job_id) for job_id in common_job_ids]

# Filter both matrices based on common users and jobs
content_scores_filtered = content_scores[user_indices_content][:, job_indices_content]
collab_scores_filtered = collab_scores[user_indices_collab][:, job_indices_collab]

hybrid_scores = (0.5 * content_scores_filtered) + (0.5 * collab_scores_filtered)
hybrid_scores = scaler.fit_transform(hybrid_scores)

In [39]:
# Step 9: Generate Recommendations
hybrid_recommendations = []
top_k = 5  # Number of job recommendations per user

for user_idx, user_id in enumerate(common_user_ids):
    user_hybrid_scores = hybrid_scores[user_idx]
    sorted_jobs = sorted(
        enumerate(user_hybrid_scores), key=lambda x: x[1], reverse=True
    )
    top_jobs = [common_job_ids[job_idx] for job_idx, score in sorted_jobs[:top_k]]
    hybrid_recommendations.append({"User ID": user_id, "Recommended Jobs": top_jobs})

hybrid_recommendations_df = pd.DataFrame(hybrid_recommendations)
hybrid_recommendations_df.to_csv("generated_recommendations.csv", index=False)

In [43]:
# Step 10: Apply Model to Testing Data

# Ensure test interaction matrix has the same columns as training
test_interaction_matrix = test_interaction_matrix.reindex(columns=train_interaction_matrix.columns, fill_value=0)

# Apply SVD transformation correctly
predicted_test_matrix = np.dot(svd.transform(test_interaction_matrix), svd.components_)

# Convert to DataFrame
predicted_df_test = pd.DataFrame(predicted_test_matrix, index=test_interaction_matrix.index, columns=train_interaction_matrix.columns)


In [48]:
# Step 11: Evaluate Model Performance – Compute Precision@K, Recall@K, NDCG, RMSE
def mean_reciprocal_rank(predictions, actual):
    reciprocal_ranks = []
    for user in predictions.index:
        if user in actual.index:
            top_predictions = predictions.loc[user].sort_values(ascending=False).index
            actual_jobs = actual.loc[user][actual.loc[user] > 0].index
            for rank, job_id in enumerate(top_predictions, start=1):
                if job_id in actual_jobs:
                    reciprocal_ranks.append(1 / rank)
                    break
    return np.mean(reciprocal_ranks) if reciprocal_ranks else 0

def precision_at_k(predictions, actual, k=5):
    precision_scores = []
    for user in predictions.index:
        if user in actual.index:
            top_k_predictions = predictions.loc[user].sort_values(ascending=False).head(k).index
            actual_jobs = actual.loc[user][actual.loc[user] > 0].index
            hits = len(set(top_k_predictions).intersection(set(actual_jobs)))
            precision_scores.append(hits / k)
    return sum(precision_scores) / len(precision_scores) if precision_scores else 0

def recall_at_k(predictions, actual, k=5):
    recall_scores = []
    for user in predictions.index:
        if user in actual.index:
            top_k_predictions = predictions.loc[user].sort_values(ascending=False).head(k).index
            actual_jobs = actual.loc[user][actual.loc[user] > 0].index
            hits = len(set(top_k_predictions).intersection(set(actual_jobs)))
            recall_scores.append(hits / len(actual_jobs) if len(actual_jobs) > 0 else 0)
    return sum(recall_scores) / len(recall_scores) if recall_scores else 0

def compute_accuracy(predictions, actual, k=5):
    correct_predictions = 0
    total_users = len(predictions.index)

    for user in predictions.index:
        if user in actual.index:
            top_k_predictions = predictions.loc[user].sort_values(ascending=False).head(k).index
            actual_jobs = actual.loc[user][actual.loc[user] > 0].index
            if len(set(top_k_predictions) & set(actual_jobs)) > 0:
                correct_predictions += 1

    return correct_predictions / total_users if total_users > 0 else 0

mrr = mean_reciprocal_rank(predicted_df_test, test_interaction_matrix)
precision = precision_at_k(predicted_df_test, test_interaction_matrix, k=5)
recall = recall_at_k(predicted_df_test, test_interaction_matrix, k=5)
ndcg = ndcg_score(test_interaction_matrix.values, predicted_df_test.values)
rmse = np.sqrt(mean_squared_error(test_interaction_matrix.values.flatten(), predicted_df_test.values.flatten()))
accuracy = compute_accuracy(predicted_df_test, test_interaction_matrix, k=5)

print(f"MRR: {mrr:.4f}")
print(f"Precision@5: {precision:.4f}")
print(f"Recall@5: {recall:.4f}")
print(f"NDCG: {ndcg:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"Accuracy: {accuracy:.4f}")

MRR: 0.8174
Precision@5: 0.2585
Recall@5: 0.9851
NDCG: 0.8564
RMSE: 0.0270
Accuracy: 0.9869


In [55]:
# Making Predictions with New Data

def get_job_recommendations_for_new_user(user_profile, job_data, tfidf, hybrid_scores, common_job_ids, top_k=5):
    """
    Generate job recommendations for a new user based on their profile.

    Args:
        user_profile (dict): New user profile containing all attributes.
        job_data (DataFrame): Job dataset.
        tfidf (TfidfVectorizer): Trained TF-IDF model.
        hybrid_scores (ndarray): Hybrid model scores (content-based + collaborative filtering).
        common_job_ids (list): List of jobs available in the hybrid model.
        top_k (int): Number of job recommendations to return.

    Returns:
        DataFrame: Top-K recommended jobs with Job ID, Job Title, and Country.
    """

    # Convert user profile into a formatted text string
    user_text = (
        user_profile['Skills'] + ' ' +
        user_profile['Interests'] + ' ' +
        user_profile['Previous Jobs'] + ' ' +
        user_profile['Looking Jobs'] + ' ' +
        user_profile['Description']
    )

    # Convert new user profile into a TF-IDF vector
    user_vector = tfidf.transform([user_text])

    # Compute similarity with jobs
    job_vectors = tfidf.transform(job_data['Details'])
    similarity_scores = cosine_similarity(user_vector, job_vectors).flatten()

    # Convert job IDs from job_data to a list
    job_ids = job_data['Job ID'].tolist()

    # Find the common job IDs between content-based and hybrid model
    common_jobs_between_models = list(set(job_ids).intersection(set(common_job_ids)))

    # Filter similarity scores to align with available hybrid model job IDs
    job_indices_content = [job_ids.index(job_id) for job_id in common_jobs_between_models]
    job_indices_hybrid = [common_job_ids.index(job_id) for job_id in common_jobs_between_models]

    similarity_scores_filtered = similarity_scores[job_indices_content]
    hybrid_scores_filtered = hybrid_scores[:, job_indices_hybrid]

    # Compute final hybrid recommendation scores
    final_scores = (0.5 * similarity_scores_filtered) + (0.5 * hybrid_scores_filtered.mean(axis=0))

    # Select top K job recommendations
    top_job_indices = final_scores.argsort()[-top_k:][::-1]
    recommended_jobs = [common_jobs_between_models[idx] for idx in top_job_indices]

    # Retrieve job details (Job ID, Job Title, Country)
    recommended_job_details = job_data[job_data['Job ID'].isin(recommended_jobs)][['Job ID', 'Job Title', 'Country']]

    return recommended_job_details

In [1]:
# Example new user profile
new_user_profile = {
    "Skills": "Waiter, Customer Service, Hospitality",
    "Interests": "Food, Service, Tourism",
    "Previous Jobs": "Hotel Staff, Receptionist",
    "Looking Jobs": "Waiter, Housekeeping, Hotel Staff",
    "Description": "Looking for a job in the hospitality industry abroad",
    "Passport Status": "Valid"
}

# Get top job recommendations for the new user
recommended_jobs = get_job_recommendations_for_new_user(
    new_user_profile, job_data, tfidf, hybrid_scores, common_job_ids, top_k=5
)

# print(f"Top job recommendations : {recommended_jobs}")

# print("\nTop job recommendations:\n")
# print(recommended_jobs.to_string(index=False))

from tabulate import tabulate

# Print nicely formatted table
print("\nTop job recommendations:\n")
print(tabulate(recommended_jobs, headers="keys", tablefmt="fancy_grid"))


NameError: name 'get_job_recommendations_for_new_user' is not defined

In [60]:
# Step 11: Save Model – Store trained models using pickle
with open("tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(tfidf, f)

with open("svd_model.pkl", "wb") as f:
    pickle.dump(svd, f)

predicted_df_test.to_pickle("predicted_matrix_test.pkl")

print("Updated model trained, tested, and saved successfully!")

Updated model trained, tested, and saved successfully!


In [61]:
from google.colab import files

# Download the saved files
files.download("tfidf_vectorizer.pkl")
files.download("svd_model.pkl")
files.download("predicted_matrix_test.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
from google.colab import files

# Upload saved models
uploaded = files.upload()

Saving predicted_matrix_test.pkl to predicted_matrix_test.pkl
Saving svd_model.pkl to svd_model.pkl
Saving tfidf_vectorizer.pkl to tfidf_vectorizer (1).pkl


In [4]:
import shutil

# Move uploaded files to a folder
shutil.move("tfidf_vectorizer.pkl", "/content/tfidf_vectorizer.pkl")
shutil.move("svd_model.pkl", "/content/svd_model.pkl")
shutil.move("predicted_matrix_test.pkl", "/content/predicted_matrix_test.pkl")

'/content/predicted_matrix_test.pkl'

In [5]:
import pickle
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

# Load pre-trained ML models
with open("/content/tfidf_vectorizer.pkl", "rb") as f:
    tfidf_vectorizer = pickle.load(f)

with open("/content/svd_model.pkl", "rb") as f:
    svd_model = pickle.load(f)

with open("/content/predicted_matrix_test.pkl", "rb") as f:
    predicted_matrix = pickle.load(f)

print("Models Loaded Successfully!")

Models Loaded Successfully!


In [42]:
# def get_new_user_recommendations(user_profile, job_data, tfidf_vectorizer, predicted_matrix, top_k=5):
#     """
#     Generate job recommendations for a new user using the trained hybrid model.
#     """

#     # Convert user profile into a formatted text string
#     user_text = (
#         user_profile['Skills'] + ' ' +
#         user_profile['Interests'] + ' ' +
#         user_profile['Previous Jobs'] + ' ' +
#         user_profile['Looking Jobs'] + ' ' +
#         user_profile['Description']
#     )

#     # Convert new user profile into a TF-IDF vector
#     user_vector = tfidf_vectorizer.transform([user_text])

#     # Compute similarity with jobs
#     job_vectors = tfidf_vectorizer.transform(job_data['Details'])
#     similarity_scores = cosine_similarity(user_vector, job_vectors).flatten()

#     # Convert Pandas Series to NumPy array before reshaping
#     similarity_scores = np.array(similarity_scores)
#     collab_scores = np.array(predicted_matrix.mean(axis=0))

#     # Scale both scores
#     scaler = MinMaxScaler()
#     similarity_scores_scaled = scaler.fit_transform(similarity_scores.reshape(-1, 1)).flatten()
#     collab_scores_scaled = scaler.fit_transform(np.array(collab_scores).reshape(-1, 1)).flatten()
#     similarity_scores_scaled = similarity_scores_scaled[:len(collab_scores_scaled)]

#     # Compute final hybrid recommendation scores
#     hybrid_scores = (0.5 * similarity_scores_scaled) + (0.5 * collab_scores_scaled)

#     # Get top job recommendations
#     top_job_indices = hybrid_scores.argsort()[-top_k:][::-1]
#     recommended_jobs = job_data.iloc[top_job_indices][['Job ID', 'Job Title', 'Country']]

#     return recommended_jobs

In [77]:
def get_new_user_recommendations(user_profile, job_data, tfidf_vectorizer, predicted_matrix, top_k=5):
    """
    Generate job recommendations for a new user using the trained hybrid model.
    """

    # Convert user profile into a formatted text string
    user_text = (
        user_profile['Skills'] + ' ' +
        user_profile['Interests'] + ' ' +
        user_profile['Previous Jobs'] + ' ' +
        user_profile['Looking Jobs'] + ' ' +
        user_profile['Description']
    )

    print(f"Job Data Length: {len(job_data)}")

    # Convert new user profile into a TF-IDF vector
    user_vector = tfidf_vectorizer.transform([user_text])

    # Compute similarity with jobs
    job_vectors = tfidf_vectorizer.transform(job_data['Details'])
    similarity_scores = cosine_similarity(user_vector, job_vectors).flatten()

    # Get collaborative filtering scores
    # collab_scores = predicted_matrix.mean(axis=0)
    collab_scores = np.zeros(len(job_data))  # Ensure correct shape

    # Fill collab_scores based on available indices
    for i in range(len(collab_scores)):
        if i < predicted_matrix.shape[1]:  # Ensure index exists in prediction matrix
            # collab_scores[i] = predicted_matrix[:, i].mean()
            collab_scores[i] = predicted_matrix.iloc[:, i].mean()

    print(f"Collaborative Scores Shape: {collab_scores.shape}")
    print(f"Similarity Scores Shape: {similarity_scores.shape}")

    # Debug: Print Top Similarity Scores
    print("\n🔹 Top Content-Based Similarity Scores:")
    sorted_indices = np.argsort(similarity_scores)[::-1][:10]
    for idx in sorted_indices:
        print(f"{job_data.iloc[idx]['Job Title']} - {similarity_scores[idx]:.4f}")

    # Debug: Print Top Collaborative Scores
    print("\n🔹 Top Collaborative Filtering Scores:")
    sorted_indices = np.argsort(collab_scores)[::-1][:10]

    # Ensure index exists in job_data before accessing it
    # for idx in sorted_indices:
    #     if idx < len(job_data):  # Prevent index out of range
    #         print(f"{job_data.iloc[idx]['Job Title']} - {collab_scores[idx]:.4f}")
    #     else:
    #         print(f"Index {idx} is out of range in job_data!")

    for idx in sorted_indices:
      if 0 <= idx < len(job_data):  # Check if index exists
          print(f"{job_data.iloc[idx]['Job Title']} - {collab_scores[idx]:.4f}")
      else:
          print(f"⚠️ Warning: Index {idx} is out of range for job_data")


    # Scale both scores
    scaler = MinMaxScaler()
    # similarity_scores_scaled = scaler.fit_transform(similarity_scores.reshape(-1, 1)).flatten()
    # collab_scores_scaled = scaler.fit_transform(collab_scores.reshape(-1, 1)).flatten()

    similarity_scores_scaled = scaler.fit_transform(similarity_scores.reshape(-1, 1)).flatten()
    collab_scores_scaled = scaler.fit_transform(np.array(collab_scores).reshape(-1, 1)).flatten()


    # similarity_scores_scaled = similarity_scores_scaled[:len(collab_scores_scaled)]
    similarity_scores_scaled = similarity_scores_scaled[:min(len(similarity_scores_scaled), len(collab_scores_scaled))]
    collab_scores_scaled = collab_scores_scaled[:min(len(similarity_scores_scaled), len(collab_scores_scaled))]



    # Compute final hybrid recommendation scores
    # hybrid_scores = (0.5 * similarity_scores_scaled) + (0.5 * collab_scores_scaled)
    # hybrid_scores = (0.7 * similarity_scores_scaled) + (0.3 * collab_scores_scaled)


    hybrid_scores = np.zeros(len(job_data))  # Ensure correct shape

    # Adjust hybrid scores only for available indices
    for i in range(len(hybrid_scores)):
        if i < len(similarity_scores_scaled) and i < len(collab_scores_scaled):
            hybrid_scores[i] = (0.7 * similarity_scores_scaled[i]) + (0.3 * collab_scores_scaled[i])

    print(f"Hybrid Scores Shape (before fix): {hybrid_scores.shape}")

    # Debug: Print Top Hybrid Scores
    print("\n🔹 Top Hybrid Recommendation Scores:")
    sorted_indices = np.argsort(hybrid_scores)[::-1][:10]
    for idx in sorted_indices:
        print(f"{job_data.iloc[idx]['Job Title']} - {hybrid_scores[idx]:.4f}")

    # Get top job recommendations
    # top_job_indices = hybrid_scores.argsort()[-top_k:][::-1]
    top_job_indices = np.argsort(hybrid_scores)[::-1][:top_k]
    recommended_jobs = job_data.iloc[top_job_indices][['Job ID', 'Job Title', 'Country']]

    return recommended_jobs

In [7]:
uploaded = files.upload()

Saving job_list.xlsx to job_list.xlsx


In [37]:
import pandas as pd

job_data = pd.read_excel("/content/job_list.xlsx")
print("Job Data Loaded!")
job_data.head()

Job Data Loaded!


,Job ID,Job Title,Country,Job Description,Skills Required,Experience Required,Age Required,Salary,Working Hours,Facilities,Looking gender,No. of job seekers required,Available Quantity
0,1,AC technicians (Auto/Domestic),Saudi Arabia,This is a job for a AC technicians (Auto/Domes...,"HVAC Systems, Electrical Troubleshooting, Cool...",Experienced,35-49,"Rs. 164,468.00",12 hours,"Accommodation, Food, Transport","male, female",104,69
1,2,AC technicians (Auto/Domestic),Lebanon,This is a job for a AC technicians (Auto/Domes...,"HVAC Systems, Electrical Troubleshooting, Cool...","1st time, Experienced",38-50,"Rs. 277,858.00",12 hours,"Accommodation, Medical, Meal Allowance","male, female",199,82
2,3,Agriculture Workers,Romania,This is a job for a Agriculture Workers in Rom...,"Crop Cultivation, Irrigation Techniques, Pest ...","1st time, Experienced",32-50,"Rs. 177,533.00",11 hours,"Accommodation, Food, Medical","male, female",26,8
3,4,Agriculture Workers,Kuwait,This is a job for a Agriculture Workers in Kuw...,"Crop Cultivation, Irrigation Techniques, Pest ...",1st time,26-55,"Rs. 229,984.00",10 hours,"Accommodation, Food, Transport","male, female",135,109
4,5,Aluminum Fabricator,Malaysia,This is a job for a Aluminum Fabricator in Mal...,"Metal Cutting, Blueprint Reading, Welding, Str...","1st time, Experienced",27-56,"Rs. 154,430.00",11 hours,"Accommodation, Food, Transport","male, female",129,18


In [46]:
# Ensure 'Details' column exists by combining relevant job attributes
job_data['Details'] = (
    job_data['Job Title'].fillna('') + ' ' +
    job_data['Skills Required'].fillna('') + ' ' +
    job_data['Job Description'].fillna('')
)

Created 'Details' column for job_data!


In [81]:
# Example new user profile
new_user_profile = {
    "User ID": 1002,
    "Name": "Mohomed Reeza",
    "Age": 30,
    "Gender": "Male",
    "Height": "5.8",
    "Weight": "70",
    "Marital Status": "Single",
    "Number of Children": "0",
    "Education": "Bachelor's Degree",
    "Skills": "cocktail mixing",
    "Interests": "liquor knowledge",
    "Previous Jobs": "Bartender",
    "Looking Jobs": "Bartender",
    "Description": "Experienced builder",
    "Passport Status": "Valid"
}

# Get top job recommendations
recommended_jobs = get_new_user_recommendations(
    new_user_profile, job_data, tfidf_vectorizer, predicted_matrix, top_k=5
)

from tabulate import tabulate

# Print nicely formatted table
print("\nTop job recommendations:\n")
print(tabulate(recommended_jobs, headers="keys", tablefmt="fancy_grid"))

Job Data Length: 200
Collaborative Scores Shape: (200,)
Similarity Scores Shape: (200,)

🔹 Top Content-Based Similarity Scores:
Bartender - 0.7768
Bartender - 0.7746
Waitress - 0.0927
Waitress - 0.0920
Painter - 0.0846
Painter - 0.0841
Mason - 0.0817
Mason - 0.0816
Auto Painter - 0.0792
Auto Painter - 0.0785

🔹 Top Collaborative Filtering Scores:
Civil Helper - 0.0150
Mechanical Helper - 0.0135
Kitchen Helper - 0.0132
Saw Mill Machine operator - 0.0112
Industrial Painter - 0.0106
Sawing Machine Operator (JUKI) - 0.0102
Driver - 0.0101
Plumber - 0.0097
Computer operators / Programmers - 0.0096
Car Washer - 0.0095
Hybrid Scores Shape (before fix): (200,)

🔹 Top Hybrid Recommendation Scores:
Bartender - 0.8498
Bartender - 0.7562
Civil Helper - 0.3000
Mechanical Helper - 0.2711
Kitchen Helper - 0.2642
Saw Mill Machine operator - 0.2239
Mason - 0.2142
Industrial Painter - 0.2120
Sawing Machine Operator (JUKI) - 0.2035
Driver - 0.2017

Top job recommendations:

╒═════╤══════════╤════════════